# Lecture 7a -- Basic Plotting and Mapping
One of the most important tools in a data scientist's arsenal is visualization and plotting. This is because visualizations are frequently the best way of communicating data to an audience and understanding it yourself. We have already seen some very basic scatter and line plots using `matplotlib.pyplot`.  In this lecture, we will also learn how customize certain aspects of our visualizations to our needs. IN this lecture, students will learn 
- How to make subplots within a single figure
- The difference between a figure and an axis
- Various properties of plots that can be customized
- What a line plot is actually doing and the benefit of using a fine `np.linspace` when constructing plots. 
- Creating maps with `GeoPandas` and `matplotlib`
- Creating interactive maps with bokeh

There are so many customization options, different plot types, etc. We won't be able to cover them all in this lecture, but you can always visit the [matplotlib documentation](https://matplotlib.org/stable/) which has plenty of examples. I also suggest QuantEcon's lecture on [Intermediate Plotting](https://datascience.quantecon.org/tools/matplotlib.html).

More advanced plotting functionality and syntax is frequently best learned needed because it can also be easy to forget unless you use it regularly. 



## Creating Effective Visaulzations in Four Steps
Here are four general steps for creating an effective visualizations:
1. **Identify your message** 
    - What is the purpose of this visualiazation? What do you want your audience to understand after seeing it?
2. **Describe your visualization**
    - What type of visualization best communicates your message? What values do you need to construct the visualization? Will this visualization be accompanied by words (either written or spoken) that can help communicate the message?
3. **Create a first draft of the visualizaiton**
    - Make a first attempt at your visualization. Does it look as you expect? If not, does it look like anything might be wrong with your data or code? If so, can you understand the message by looking at it? How about a stranger?
4. **Iterate and improve upon your visualization**
    - How can you make the visualization more informative and aesthetically appealing? This is where you tinker with colors, alignments, labels, titles, legends, tick marks etc. to find the optimal visualization. 

We won't discuss examples of good and bad visualization in this lecture.  There are lecture notes on ["Data Visualization Rules and Guidelines"](https://datascience.quantecon.org/tools/visualization_rules.html) on QuantEcon. I highly suggest you read them when you get a chance.



## `matplotlib` and `pyplot`
`matplotlib` is a popular plotting and visualization package in Python. In past examples and for this lecture, we will be using the sub-module `pyplot`. Below, we import `matplotlib.pyplot` as well as `numpy` using the standard aliases. We will also import `math` for use later. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS

## `plt.subplots`
Up until this point, we have beeusing `plt.plot` which is probably the simplest plotting function.  Now, we demonstrate `plt.subplots()` which gives us more customization over our visualization and allows us to have many axes within one figure.  It is best to learn through example. Below, we plot the function 
$y = sin(x) + |x|$ for $x$ in  $[-4\pi,4 \pi]$.

In [ ]:
# Initialize plot -- subplots returns two objects, a figure and axes
fig, ax = plt.subplots()

# Next we use np.linspace() to create our x values
x = np.linspace(-4 * np.pi, 4 * np.pi, 100)
# Generate y
y = np.sin(x) + np.abs(x) 

# Plug the values into our plot
ax.plot(x, y)

## Figures and Axes
In the code above, initializaing `plt.subplots()` returned a figure object and an axis object which we called `fig` and `ax` respectively. QuantEcon uses the analogy of a framed painting. 
- The axis is is the canvas on which we draw plots 
- The figure is the entire framed painting, including the axis itself.

We can actually change the color of both of these objects and observe what happens.

In [ ]:
fig, ax = plt.subplots()

fig.set_facecolor("green") # This makes the edges green
ax.set_facecolor("purple") # This makes the "canvas" purple

## Multiple Axes
This figure/axis separation allows us to display many  axes on a single figure. 
Below, we plot many sine waves of varying frequencies on the same figure. 

In [ ]:
fig, axes = plt.subplots(2, 2) # This creates a figure with 4 axes arranged in a 2x2 grid
x_val = np.linspace(-2 * np.pi, 2 * np.pi, num = 1000) # create set of x values
freq = np.array([.25, .5, 1, 2]) # various frequencies
vec_sin = np.vectorize(np.sin) # vectorize sin function to apply to vectors


for i, ax in enumerate(axes.flat): # Loops through each axis to add the lines
    ax.plot(x_val, vec_sin(freq[i] * 2 * np.pi* x_val)) # plot red line on each individual axis


## Making Your Plots Look Nice
So far, we have created very bare bones figures, but `pyplot` allows for a lot of additional customization. For example, we might want to include titles in the plots above, an x-axis label, or y-axis labels. We can even vary the colors of the lines!

In [ ]:
plot_colors = ["red", "orange", "purple", "green"]
for i, ax in enumerate(axes.flat):
    ax.plot(x_val, vec_sin(freq[i] * 2 * np.pi* x_val), color = plot_colors[i])
    ax.set_title("Sine Wave with Freq=" + str(freq[i])) 
    ax.set_xlabel("x") 
    ax.set_ylabel("y") 
    ax.set_label(freq[i])

fig

## Overlapping Text
Unfortunately, all of the titles and x-axis labels we added overlap with the axis tik marks. To fix this, we can simply call `fig.tight_layout()` to make sure all labels and other text do not overlap with eachother. 

In [ ]:
fig.tight_layout()
fig

## Coarsness of `np.linspace()`
When creating line plots of these functions, we are actually plotting many points and connecting them with lines. The x values are generated by a `np.linspace()` call and the y values are generated by taking some function (such as `np.sin()`) of those x values. Generally speaking, the finer our grid of points generated by `np.linspace()`, the more our plot will resemble the true function.

Below, we consider coarse and finer grid values. We also illustrate the use of `label` and `legend` to label lines. 

In [ ]:
x_fine = np.linspace(-2 * np.pi, 2 * np.pi, num = 100)
x_coarse = np.linspace(-2 * np.pi, 2 * np.pi, num = 10)

y_fine = np.sin(x_fine)
y_coarse = np.sin(x_coarse)

fig, ax = plt.subplots()
ax.plot(x_coarse, y_coarse, label = "coarse line")
ax.plot(x_fine, y_fine, label = "fine line")
fig.legend()
ax.set_title("Coarse vs. Fine Sine Function")

This plot makes it clear that a fine grid will generally result in a better looking curve. If we look at scatter plots instead of line plots, we can see the points that the line plot is interpolating between. 

In [ ]:
fig, ax = plt.subplots(2)
ax[0].scatter(x_coarse, y_coarse, label = "coarse points")
ax[0].plot(x_coarse, y_coarse, label = "coarse points")
ax[1].scatter(x_fine, y_fine, label = "fine points")
ax[1].plot(x_fine, y_fine, label = "fine points")
ax[0].set_title("Coarse vs. Fine Sine Si Function")

## Saving Plots
Sometimes, we want to save plots as image files so we can use the figures we create in other applications. Using the `.savefig()` method on a plot object. use the suffix to specify the filetype. 

In [ ]:
pwd()

In [ ]:
fig.savefig("..//lecture_generated_objects//coarse_fine_sine.png")
fig.savefig("..//lecture_generated_objects//coarse_fine_sine.jpeg")

## Mapping
Now we are going to switch gears and talk about mapping in Python. To do this, we will use the `geopandas` package. GeoPandas makes it easier to process geospatial data. While geospatial data is used for many purposes, our ultimate goal is to visualize geospatial data using maps. 

This section focuses on the same material as QuantEcon's lecture on [Mapping in Python](https://datascience.quantecon.org/tools/maps.html) but with different examples. 


In this lecture, our ultimate goal will be to have a plot of Canada and the markers/labels for the following nine cities in Canada.


In [ ]:
## City Info
cities = ['Toronto', 'Montreal', 'Calgary', 'Edmonton', 'Ottawa', 'Winnipeg',
             'Vancouver',  'Halifax']
provinces = ['Ontario', 'Quebec', 'Alberta', 'Alberta', 'Ontario', 'Manitoba',
             'British Columbia',  'Nova Scotia']
df = pd.DataFrame({
    'City': cities,
    'Country': 8*["Canada"],
    'Province': provinces,
    'Latitude': np.array([43.7, 45.509, 51.05, 53.55, 45.411, 49.884, 
                  49.25,  44.643]),
    'Longitude': np.array([-79.416, -73.588, -114.085, -113.469, -75.698, -97.147, 
                  -123.119, -63.577])
})

We can use `zip()` to generate a column that contains longitude and latitude as a coordinate pair. 

In [ ]:
df["Coordinates"] = list(zip(df.Longitude, df.Latitude))
df.head()

### `Shapely Point` Object
Currently, the coordinates have the type tuple. To use these points later, we need to turn them into a `Shapely Point` object using the `.apply()` method with the argumenet `Point`. 


In [ ]:
df["Coordinates"] = df["Coordinates"].apply(Point)
df.head()

### `DataFrame` to `GeoDataFrame`
Finally, we have to convert this DataFrame into a GeoDataFrame. GeoDataFrames are DataFrames with added functionality that makes it easier to plot maps. We use the `gpd` function `GeoDataFrame(d)` to convert the DataFrame. We must supply an argument called `geometry` that tells GeoPandas which column should treat as a spatial attribute. 

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry="Coordinates")
gdf.head()

Can check its type and figure out which column is the geometry column

In [ ]:
print(type(gdf))
print(gdf.geometry.name)

## Still Need a Map
Our goal is to plot the locations of our cities on a map of Canada. First, we must obtain and plot the map. Then, we can plot the cities. 



### Obtaining a Canadian Map
To get the map, we need to find shapefiles that contain the geometric data needed to plot the contours of a geographic region. I Googled "Canadian Province Shapefile GeoPandas" and found that [Statistics Canada](https://www.statcan.gc.ca/en/start) has shapefiles for download [here](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm). This file is also in the course repository. This is a good approach to finding shapefiles for other geographical objects. 

In [ ]:
can_df = gpd.read_file("..//data//canada_shapes//lpr_000b16a_e.shp")
can_df

### Not all shapefiles use latitude and longitude.
Look at the multipolygon coordinates. Do those look like latitude and longitude? If units do not match, our cities won't line up with the Canadian map. Below, we convert the polygon units so they use latitude and longitude.

In [ ]:
can_df = can_df.to_crs(CRS.from_string('EPSG:4326'))
can_df.head()

## Time to Plot
 Now, all we need to is plot the city data in our already existing plot. `gdf` 

In [ ]:
# This line initializes the figure
fig, gax = plt.subplots(figsize = (10,15)) 
# Plot the map
can_df.plot(ax=gax, edgecolor='black', color='aliceblue') 
# Plot the cities
gdf.plot(ax=gax, color='red', alpha = .5, markersize = 60) 

# Sets the Labels
gax.set_xlabel('longitude')
gax.set_ylabel('latitude')

# remove top and right edges
gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)


### One Step Further -- Fill, Labels, and Polish
Let's take this map one step further. First, we show how we can use color to communicate numerical information. To do this, we will color provinces according to their populations. We will also add labels to the cities polish other aspects of the plot.

First we have to load data on populations into our notebook and join it with the GeoDataFrame we've already created. 

In [ ]:
pop_df = pd.read_csv("..//data//province_pop.csv")
pop_df

We can join this with our `GeoDataFrame` using `pd.merge()`.

In [ ]:
# Merge with geodata
pop_df = pd.merge(pop_df, can_df, left_on = "Geography", right_on = "PRENAME")

# Print type -- it's a normal DataFrame
print(type(pop_df))

# Make it a GeoDataFrame (doesn't like pre-existing column called "geometry")
pop_df = pop_df.rename(columns = {'geometry':'polygon'})
pop_df = gpd.GeoDataFrame(pop_df, geometry = "polygon")


In [ ]:
pop_df

In [ ]:
# Initialize the Plot
fig, gax = plt.subplots(figsize = (10,15)) # This line initializes the figure

# Plot the map with fill
pop_df.plot(ax=gax, edgecolor='grey',  column = "Q12023", cmap = "Wistia", legend = True, legend_kwds={'shrink': 0.3})
# Plot Points
gdf.plot(ax=gax, color='blue', alpha = .5, markersize = 60) # plot the cities

# Plot Labels -- position labels differently for different cities
for x, y, label in zip(gdf['Coordinates'].x, gdf['Coordinates'].y, gdf['City']):
    if label not in ["Montreal", "Halifax", "Toronto", "Ottawa", "Vancouver"]:
        gax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points')
    elif label in ["Montreal", "Halifax", "Toronto"]:
        gax.annotate(label, xy=(x,y), xytext=(4,-6), textcoords='offset points')
    elif label in ["Ottawa"]:
        gax.annotate(label, xy=(x,y), xytext=(-8,4), textcoords='offset points')
    elif label in ["Vancouver"]:
        gax.annotate(label, xy=(x,y), xytext=(-10,4), textcoords='offset points')

# Sets the Labels
gax.set_xlabel('longitude')
gax.set_ylabel('latitude')

# Add a Title
gax.set_title("Q1 2023 Population by Province")

# remove top and right edges
gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)

fig.tight_layout()


## Saving Maps
Just like with plots, we can use `.savefig()` method on figures that contain maps. 

In [ ]:
fig.savefig("..\\lecture_generated_objects//canada_polished.png")

## Interactive Maps
Usinkg the `bokeh` library, we can even make our maps interactive! Now, we can hover over geographic objects to see information about them!

In [ ]:
# Import various functions 
from bokeh.io import output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, save
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer

# Shows plots in notebook
output_notebook()

# lets us convert geodf to json format
import json

#Convert pop_data to geojson for bokeh
pop_geojson=GeoJSONDataSource(geojson=pop_df.to_json())
# turn cities into json for bokeh
gdf_geojson=GeoJSONDataSource(geojson=gdf.to_json()) 

In [ ]:
# Specify the color gradient that represent numeric values
color_mapper = LinearColorMapper(palette = "Cividis256", low = 0, high = pop_df["Q12023"].max())
# color bar at bottom
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')

# create figure
p = figure(title="Q1 2023 Population by Province")


In [ ]:
# This line will take a long time to run, likely due to high resolution of file
plot_pop = p.patches("xs","ys",source=pop_geojson,
          fill_color = {'field' :'Q12023', 'transform' : color_mapper})
p.add_tools(HoverTool(renderers = [plot_pop], tooltips = [ ('Province','@Geography'),('Q1 2023 Pop', '@Q12023'),
                               ('Q2 2023 Pop','@Q22023')]))


plot_city = p.circle(x="Longitude", y="Latitude", size=10, fill_color="orange", fill_alpha=0.8, source=gdf_geojson, )
p.add_tools(HoverTool(renderers = [plot_city], tooltips = [ ('City',"@City"), ('Province','@Province')]))
p.add_layout(color_bar, 'below')
show(p)